In [1]:
import re
import pandas as pd

In [22]:
data=[]
with open ("access.log", "r") as myfile:
    for line in myfile:
        data.append(list(map(''.join, re.findall(r'\"(.*?)\"|\[(.*?)\]|(\S+)', line))))
df = pd.DataFrame(data)
df=df.drop(columns=[1,2,6,7])
df.columns=['Ip','Time','Type','Rcode','Com']
wl=['109.171.82.22']
bl=['92.94.108.47']
df.head()

,Ip,Time,Type,Rcode,Com
0,185.49.14.190,21/Sep/2015:08:57:09 +0200,GET http://testp5.mielno.lubin.pl/testproxy.ph...,499,Mozilla/5.0 (Windows NT 5.1; rv:32.0) Gecko/20...
1,54.224.20.199,21/Sep/2015:11:01:51 +0200,GET / HTTP/1.1,499,Cloud mapping experiment searching for shoutca...
2,54.224.20.199,21/Sep/2015:11:01:51 +0200,HEAD / HTTP/1.1,499,Cloud mapping experiment. Contact research@pdr...
3,141.135.99.155,21/Sep/2015:13:07:45 +0200,HEAD /robots.txt HTTP/1.0,499,-
4,144.76.78.194,21/Sep/2015:13:55:35 +0200,GET / HTTP/1.1,499,Python-urllib/2.7


In [25]:
#Условия подозрительности:
#Список белых Ip 
#Список черных Ip
#код 499
#кто пытался подключиться к php my admin кроме 109.171.82.22
#кто пытался подключиться к wp-login кромe 109.171.82.22
def susp(r):
    k=0
    if r['Rcode']==499:
        k+=1
    if r['Ip']not in wl:
        k+=1
    if r['Ip']in bl:
        k+=1
    a = re.search(r'wp-login*', r['Type'])
    if a is not None:
        if r['Ip']not in wl:
            k+=1
    a = re.search(r'phpmyadmin*', r['Type'])
    if a is not None:
        if r['Ip']not in wl:
            k+=1    
    if k >= 2:
        return True
    else:
        return False
df['susp'] = df.apply(lambda x: susp(x), axis=1)
df.head()

,Ip,Time,Type,Rcode,Com,susp
0,185.49.14.190,21/Sep/2015:08:57:09 +0200,GET http://testp5.mielno.lubin.pl/testproxy.ph...,499,Mozilla/5.0 (Windows NT 5.1; rv:32.0) Gecko/20...,False
1,54.224.20.199,21/Sep/2015:11:01:51 +0200,GET / HTTP/1.1,499,Cloud mapping experiment searching for shoutca...,False
2,54.224.20.199,21/Sep/2015:11:01:51 +0200,HEAD / HTTP/1.1,499,Cloud mapping experiment. Contact research@pdr...,False
3,141.135.99.155,21/Sep/2015:13:07:45 +0200,HEAD /robots.txt HTTP/1.0,499,-,False
4,144.76.78.194,21/Sep/2015:13:55:35 +0200,GET / HTTP/1.1,499,Python-urllib/2.7,False


In [28]:
susp_zapr = df[df["susp"] == True]
susp_zapr['Ip'].unique()

array(['144.76.78.194', '5.135.213.197', '92.94.108.47', '94.23.74.168'],
      dtype=object)